In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

# Strategy we will backtest

#### 1. At 10:59 AM calculate the day high and day Low.
    
    
#### 2. From 11 AM keep monitoring the stock price.


#### 3. If it breaches the Low enter the trade with Short


#### 4. If it breaches the high enter the trade with Long


#### 5. If neither of #4 or #5 happens, don't enter.


#### 6. Once, trade entered. Set yourself a target and a stop loss.


#### 7. Exit the trade when either target or stop-loss is breached.

## Step 0 - Declaring the Parameters

In [ ]:
# To be used in Profit and Loss calculcation
brokerage = 40

# Quantity of stocks in 1 Lot of Bank Nifty
lot_size = 25

# This is a factor we are including for slippage and other tax deductions
# This variable is to incorporate STT deductions
adjustment_factor = 0.95 

# Interval for which we calucalate our High and Low
interval_dict = {"10 AM" :  45,
                 "11 AM" :  105,
                 "12 PM" :  165,
                 "1 PM"  :  225,
                 "2 PM"  :  285}


interval_in_mins = interval_dict['11 AM']

start_time = '10:59' #change it as per interval selection
end_time = '15:00'

#Target Points
target_param = 100

# Stop Loss Points
loss_param = 175

# This date is an outlier. On this day Trading happens at a different time-frame.
muhurat_trading_day = '04-11-2021' 


#File Path
#data_path = '/Users/sreemantakesh/Desktop/The Algotrading Page/Datasets/FUTURES/BANK NIFTY FUT 2021.csv'
data_path = 'https://raw.githubusercontent.com/Sreemanto/SK__007/master/AlgoTrading%20Workshop/BANK%20NIFTY%20FUT%202021.csv'

## Step 1 - Load the data and perform elementary checks

#### 1. No missing values.

#### 2. Check the data-types

In [ ]:
df = pd.read_csv(data_path)
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

# Step 2 - Create two columns time, and day. Such that

#### 'Day' = DD-MM-YYYY
#### 'Time' = HH:MM
    
Also, convert, the 'date' column to 'datetime' object.

In [ ]:
df['time']= df.date.str.slice(10,16)

df['day'] = df.date.str.slice(0,10)

df["date"] = pd.to_datetime(df["date"])

df.head(5)

# Step 3 - At 10:59 AM calculate the day high and day Low  for each day and store them as columns with name as 'interval_high' and 'interval_low'. 

In [ ]:
#doing a groupby and only keeping the first 105 entries. Why?
temp = df.groupby('day').head(interval_in_mins)
temp.head(5)

In [ ]:
interval_high_low = temp.groupby('day', as_index=False).agg({"high":"max", "low":"min"})
interval_high_low.head(5)

In [ ]:
#renaming the columns
interval_high_low.columns = ['day','interval_high','interval_low']
interval_high_low.head(5)

In [ ]:
#Joining with the original dataframe 'df'
df = pd.merge(df, interval_high_low, on = 'day')

df.head(5)

# Step 4 - Filter the data for just one day '04-01-2021' . And do the following.

#### 1. Store the 11 AM high and 11 AM low in variables 'interval_high' and 'interval_low'

#### 2. Store the 11 AM high and 11 AM low in variables 'interval_high_time' and 'interval_low_time'

#### 3. Filter the data further such that we only have data from 11 AM to 3 PM for the given day

In [ ]:
day = '04-01-2021'

#filtering the data for one single day
d1 = df[df["day"] == day]


In [ ]:
#Storing the 11 AM high and 11 AM low in variables 'interval_high' and 'interval_low'
interval_high = d1['interval_high'].unique()[0]
interval_low = d1['interval_low'].unique()[0]

#Storing the 11 AM high and 11 AM low in variables 'interval_high_time' and 'interval_low_time'
interval_high_time = d1[d1['high']==interval_high].date.dt.time.values[0]
interval_low_time = d1[d1['low']==interval_low].date.dt.time.values[0]

In [ ]:
#subsetting the data for post to interval time
d1 = d1.set_index('date')

d1 = d1.between_time(start_time,end_time)

d1 = d1.reset_index()

d1.head(5)

# Step 5 - Check if trade is triggered or not.

#### 1. Create two boolean columns 'interval_high_breach' and 'interval_low_breach' such that position after the high breach or low breach are all marked as 1.

#### 2. Declare and empty variable name it as 'position'

#### 3. Check if trade is triggered or not. If it breaks high than store position = 'Long', if low is breached then store as 'Short' else "NA"

#### 4. Store the 'row number' or the index of the dataframe where the trade was triggered in a variable 'entry_index' or 'entry_index'

#### 4. Also, store the entry_price, target, stop_loss, entry_time for all the three cases

In [ ]:
position = ''

d1['interval_high_breach'] = np.where(d1.high > interval_high , True, False)
d1['interval_low_breach'] = np.where(d1.low < interval_low , True, False)

d1[['time', 'day', 'open', 'high', 'low', 'close','time',
    'day', 'interval_high','interval_low', 'interval_high_breach', 'interval_low_breach']]

In [ ]:
#Condtion 1 - When after 11 AM both high breach and low breack has happend on that day
if len(d1[d1['interval_low_breach'] == True]) & len(d1[d1['interval_high_breach'] == True]):
    
    long_entry_index = d1[d1['interval_high_breach'] == True].index[0]
    short_entry_index = d1[d1['interval_low_breach'] == True].index[0]


    if long_entry_index < short_entry_index:

        position = 'Long'
        
        entry_price = interval_high
        
        entry_index = d1[d1['interval_high_breach'] == True].index[0]
        
        target    = entry_price + target_param
        
        stop_loss = entry_price - loss_param
        
        entry_time = d1['date'].dt.time[entry_index]    
        
        
    else:

        position = 'Short'
        
        entry_price = interval_low
        
        entry_index = d1[d1['interval_low_breach'] == True].index[0]
        
        target    = entry_price - target_param
        
        stop_loss = entry_price + loss_param
        
        entry_time = d1['date'].dt.time[entry_index]
        
#Condtion 2 - When after 11 AM only low breach has happened
elif len(d1[d1['interval_low_breach'] == True]):

    position = 'Short'

    entry_price = interval_low

    entry_index = d1[d1['interval_low_breach'] == True].index[0]

    target    = entry_price - target_param

    stop_loss = entry_price + loss_param

    entry_time = d1['date'].dt.time[entry_index]

#Condtion 3 - When after 11 AM only high breach has happened
elif len(d1[d1['interval_high_breach'] == True]):

    position = 'Long'

    entry_price = interval_high

    entry_index = d1[d1['interval_high_breach'] == True].index[0]

    target    = entry_price + target_param

    stop_loss = entry_price - loss_param

    entry_time = d1['date'].dt.time[entry_index]   
        
else:

    position = 'NA'
    
    entry_price = np.nan

    entry_index = np.nan

    target    = np.nan

    stop_loss = np.nan

    entry_time = np.nan

In [ ]:
print("Trade triggered  position entered ",position)
print("Trade entered at : ",format(entry_time))
print("Entry Price : ",entry_price)
print("Traget : ",target)
print("Stop Loss : ",stop_loss)

# Step 6 : Check the outcome of the Trade
    
#### 1. Declare two empty variables 'closing_price' and 'exit_time'

#### 2. Loop through the data after the trade has entered to check for profit or loss.

#### 3. Store the outcome in variable called 'result'. Also store the exit time and closing price 

In [ ]:
closing_price = ''
exit_time = ''

if position!="NA":

    post_signal_data = d1.iloc[entry_index:,:][["high","low","close","time"]].reset_index(drop=True).copy()        

    #iterate and check if target hits or stop loss hits
    if position == "Long":
        for candle_high, candle_low, closing,time in zip(post_signal_data.high,
                                                         post_signal_data.low,
                                                         post_signal_data.close,
                                                         post_signal_data.time):
            if candle_high > target:
                result = 'Profit'
                #print('Profit')
                closing_price = target
                exit_time = time
                break
            elif candle_low < stop_loss:
                result = 'Loss'
                #print('Loss')
                closing_price = stop_loss
                exit_time = time
                break
            else:
                result = "Started but not finished"
                closing_price = closing
                exit_time = time

    else:
        for candle_high, candle_low, closing, time in zip(post_signal_data.high,
                                                          post_signal_data.low,
                                                          post_signal_data.close,
                                                          post_signal_data.time):
            if candle_low < target:
                result = 'Profit'
                closing_price = target
                exit_time = time
                #print('Profit')
                break
            elif candle_high > stop_loss:
                result = 'Loss'
                closing_price = stop_loss
                exit_time = time
                #print('Loss')
                break
            else:
                result = "Started but not finished"
                last_price = post_signal_data
                closing_price = closing
                exit_time = time
else:
    result = "NA"
    closing_price = 'NA'
    
# Copy the data in a list
rpt = []

rpt.append((day,
            interval_high,
            interval_high_time,
            interval_low,
            interval_low_time,
            position,
            entry_time,
            exit_time,
            result,
            entry_price,
            closing_price))

In [ ]:
print("Trade triggered {} position entered ".format(position))
print("Trade entered at : ",format(entry_time))
print("Entry Price : ",entry_price)
print("Traget : ",target)
print("Stop Loss : ",stop_loss)
print("Trade Result : ",result)
print("Trade Exited at : ",exit_time)

# Step 7 : Store the Result in the form of a DataFrame


In [ ]:
rpt_df = pd.DataFrame(rpt,columns= ['day',
                'interval_high',
                'interval_high_time',
                'interval_low',
                'interval_low_time',
                'position',
                'entry_time',
                'exit_time',
                'result',
                'entry_price',
                'closing_price'])

rpt_df